# Using Finite State Machines

While Object-Oriented Programming provides a strong perspective of most situations, classes can be a poor tool for modelling the real-world if one object may act differently under specific scenarios.  In such cases, programmers tend to overly use conditionals to force the models' correct behaviour.

An alternative is finite state machines.  This notebook investigates finite state machines, when they are useful, and how to implement them.

In [1]:
let account = {
  state: 'open',
  balance: 0,

  deposit (amount) {
    if (this.state === 'open') {
      this.balance = this.balance + amount;
    } else {
      throw 'invalid event';
    }
  },

  withdraw (amount) {
    if (this.state === 'open') {
      this.balance = this.balance - amount;
    } else {
      throw 'invalid event';
    }
  },

  close () {
    if (this.state === 'open') {
      if (this.balance > 0) {
        // ...transfer balance to suspension account
      }
      this.state = 'closed';
    } else {
      throw 'invalid event';
    }
  },

  reopen () {
    if (this.state === 'closed') {
      // ...restore balance if applicable
      this.state = 'open';
    } else {
      throw 'invalid event';
    }
  }
}


In [2]:
account.state  //=> open

'open'

In [3]:
account.close();
account.state

'closed'

In [1]:
let account = {
  state: 'open',
  balance: 0,

  deposit (amount) {
    if (this.state === 'open' || this.state === 'held') {
      this.balance = this.balance + amount;
    } else {
      throw 'invalid event';
    }
  },

  withdraw (amount) {
    if (this.state === 'open') {
      this.balance = this.balance - amount;
    } else {
      throw 'invalid event';
    }
  },

  placeHold () {
    if (this.state === 'open') {
      this.state = 'held';
    } else {
      throw 'invalid event';
    }
  },

  removeHold () {
    if (this.state === 'held') {
      this.state = 'open';
    } else {
      throw 'invalid event';
    }
  },

  close () {
    if (this.state === 'open' || this.state === 'held') {
      if (this.balance > 0) {
        // ...transfer balance to suspension account
      }
      this.state = 'closed';
    } else {
      throw 'invalid event';
    }
  },

  reopen () {
    if (this.state === 'closed') {
      // ...restore balance if applicable
      this.state = 'open';
    } else {
      throw 'invalid event';
    }
  }
}


In [2]:
account.state  //=> open

'open'

In [3]:
account.close();
account.state

'closed'

The principle is that instead of using strings for state, we’ll use objects that contain the methods we’re interested in. 

In [ ]:
const STATE = Symbol("state");
const STATES = Symbol("states");

const open = {
  deposit (amount) { this.balance = this.balance + amount; },
  withdraw (amount) { this.balance = this.balance - amount; },
  placeHold () {
    this[STATE] = this[STATES].held;
  },
  close () {
    if (this.balance > 0) {
      // ...transfer balance to suspension account
    }
    this[STATE] = this[STATES].closed;
  }
};

const held = {
  removeHold () {
    this[STATE] = this[STATES].open;
  },
  deposit (amount) { this.balance = this.balance + amount; },
  close () {
    if (this.balance > 0) {
      // ...transfer balance to suspension account
    }
    this[STATE] = this[STATES].closed;
  }
};

const closed = {
  reopen () {
    // ...restore balance if applicable
    this[STATE] = this[STATES].open;
  }
};


Now our actual account object stores a state object rather than a state string, and delegates all methods to it. When an event is invalid, we’ll get an exception. That can be “fixed,” but let’s not worry about it now:

In [ ]:
const account = {
  balance: 0,

  [STATE]: open,
  [STATES]: { open, held, closed },

  deposit (...args) { return this[STATE].deposit.apply(this, args); },
  withdraw (...args) { return this[STATE].withdraw.apply(this, args); },
  close (...args) { return this[STATE].close.apply(this, args); },
  placeHold (...args) { return this[STATE].placeHold.apply(this, args); },
  removeHold (...args) { return this[STATE].removeHold.apply(this, args); },
  reopen (...args) { return this[STATE].reopen.apply(this, args); }
};


In [8]:
STATE = Symbol("state");
STATES = Symbol("states");

function transitionsTo (stateName, fn) {
  return function (...args) {
    const returnValue = fn.apply(this, args);
    this[STATE] = this[STATES][stateName];
    return returnValue;
  };
}

open = {
  deposit (amount) { this.balance = this.balance + amount; },
  withdraw (amount) { this.balance = this.balance - amount; },
  placeHold: transitionsTo('held', () => undefined),
  close: transitionsTo('closed', function () {
    if (this.balance > 0) {
      // ...transfer balance to suspension account
    }
  })
};

held = {
  removeHold: transitionsTo('open', () => undefined),
  deposit (amount) { this.balance = this.balance + amount; },
  close: transitionsTo('closed', function () {
    if (this.balance > 0) {
      // ...transfer balance to suspension account
    }
  })
};

closed = {
  reopen: transitionsTo('open', function () {
    // ...restore balance if applicable
  })
};

account = {
  balance: 0,

  [STATE]: open,
  [STATES]: { open, held, closed },

  deposit (...args) { return this[STATE].deposit.apply(this, args); },
  withdraw (...args) { return this[STATE].withdraw.apply(this, args); },
  close (...args) { return this[STATE].close.apply(this, args); },
  placeHold (...args) { return this[STATE].placeHold.apply(this, args); },
  removeHold (...args) { return this[STATE].removeHold.apply(this, args); },
  reopen (...args) { return this[STATE].reopen.apply(this, args); }
};


{ balance: 0,
  deposit: [Function: deposit],
  withdraw: [Function: withdraw],
  close: [Function: close],
  placeHold: [Function: placeHold],
  removeHold: [Function: removeHold],
  reopen: [Function: reopen],
  [Symbol(state)]: 
   { deposit: [Function: deposit],
     withdraw: [Function: withdraw],
     placeHold: [Function],
     close: [Function] },
  [Symbol(states)]: 
   { open: 
      { deposit: [Function: deposit],
        withdraw: [Function: withdraw],
        placeHold: [Function],
        close: [Function] },
     held: 
      { removeHold: [Function],
        deposit: [Function: deposit],
        close: [Function] },
     closed: { reopen: [Function] } } }

In [6]:
account.state  //=> open

In [10]:
const RESERVED = [STARTING_STATE, STATES];

function StateMachine (description) {
  const machine = {};

  // Handle all the initial states and/or methods
  const propertiesAndMethods = Object.keys(description).filter(property => !RESERVED.includes(property));
  for (const property of propertiesAndMethods) {
    machine[property] = description[property];
  }

  // now its states
  machine[STATES] = description[STATES];

  // what event handlers does it have?
  const eventNames = Object.entries(description[STATES]).reduce(
    (eventNames, [state, stateDescription]) => {
      const eventNamesForThisState = Object.keys(stateDescription);

      for (const eventName of eventNamesForThisState) {
        eventNames.add(eventName);
      }
      return eventNames;
      },
    new Set()
  );

  // define the delegating methods
  for (const eventName of eventNames) {
    machine[eventName] = function (...args) {
      const handler = this[STATE][eventName];
      if (typeof handler === 'function') {
        return this[STATE][eventName].apply(this, args);
      } else {
        throw `invalid event ${eventName}`;
      }
    }
  }

  // set the starting state
  machine[STATE] = description[STATES][description[STARTING_STATE]];

  // we're done
  return machine;
}


In [11]:
STATES = Symbol("states");
STARTING_STATE = Symbol("starting-state");

function transitionsTo (stateName, fn) {
  return function (...args) {
    returnValue = fn.apply(this, args);
    this[STATE] = this[STATES][stateName];
    return returnValue;
  };
}

account = StateMachine({
  balance: 0,

  [STARTING_STATE]: 'open',
  [STATES]: {
    open: {
      deposit (amount) { this.balance = this.balance + amount; },
      withdraw (amount) { this.balance = this.balance - amount; },
      placeHold: transitionsTo('held', () => undefined),
      close: transitionsTo('closed', function () {
        if (this.balance > 0) {
          // ...transfer balance to suspension account
        }
      })
    },
    held: {
      removeHold: transitionsTo('open', () => undefined),
      deposit (amount) { this.balance = this.balance + amount; },
      close: transitionsTo('closed', function () {
        if (this.balance > 0) {
          // ...transfer balance to suspension account
        }
      })
    },
    closed: {
      reopen: transitionsTo('open', function () {
        // ...restore balance if applicable
      })
    }
  }
});


{ balance: 0,
  deposit: [Function],
  withdraw: [Function],
  placeHold: [Function],
  close: [Function],
  removeHold: [Function],
  reopen: [Function],
  [Symbol(states)]: 
   { open: 
      { deposit: [Function: deposit],
        withdraw: [Function: withdraw],
        placeHold: [Function],
        close: [Function] },
     held: 
      { removeHold: [Function],
        deposit: [Function: deposit],
        close: [Function] },
     closed: { reopen: [Function] } },
  [Symbol(state)]: 
   { deposit: [Function: deposit],
     withdraw: [Function: withdraw],
     placeHold: [Function],
     close: [Function] } }

In [12]:
account.balance

0

In [13]:
account.close()

In [17]:
account.state

In [15]:
account.reopen()

In [16]:
account.balance

0